In [8]:
import tensorflow as tf

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import plotly.graph_objects as go
from prettytable import PrettyTable

mpl.rcParams['figure.figsize'] = (12, 6)
mpl.rcParams['axes.grid'] = False

In [9]:
df = pd.read_csv("../demand-forecasting-in-retail/train.csv")
df

,Unnamed: 0,product_rk,store_location_rk,period_start_dt,demand,PROMO1_FLAG,PROMO2_FLAG,PRICE_REGULAR,PRICE_AFTER_DISC,NUM_CONSULTANT,AUTORIZATION_FLAG
0,0,40369,309,2016-12-19,29.0,NaN,NaN,NaN,NaN,NaN,NaN
1,1,40370,309,2016-12-19,64.0,NaN,NaN,NaN,NaN,NaN,NaN
2,2,40372,309,2016-12-19,32.0,NaN,NaN,NaN,NaN,NaN,NaN
3,3,40373,309,2016-12-19,10.0,NaN,NaN,NaN,NaN,NaN,NaN
4,4,46272,309,2016-12-19,15.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
35339,35537,40370,1380,2019-12-30,NaN,0.0,0.0,1000.00,1000.0,0.0,1.0
35340,35538,40372,1380,2019-12-30,NaN,0.0,0.0,2000.00,2000.0,0.0,1.0
35341,35539,40373,1380,2019-12-30,NaN,0.0,0.0,3000.00,3000.0,0.0,1.0
35342,35540,46272,1380,2019-12-30,NaN,1.0,0.0,284.29,199.0,0.0,1.0


In [10]:
df['date'] = pd.to_datetime(df.period_start_dt, format='%Y-%m-%d')
# log price
# df['PRICE_REGULAR'] = np.log(df.PRICE_REGULAR)
# date to 10^8 sec
df.date = df.date.astype(np.int64)/3600/24
df.date = df.date.astype(np.int64)/10**9

In [11]:
period_starts = df.period_start_dt.unique()
period_starts

array(['2016-12-19', '2016-12-26', '2017-01-02', '2017-01-09',
       '2017-01-16', '2017-01-23', '2017-01-30', '2017-02-06',
       '2017-02-13', '2017-02-20', '2017-02-27', '2017-03-06',
       '2017-03-13', '2017-03-20', '2017-03-27', '2017-04-03',
       '2017-04-10', '2017-04-17', '2017-04-24', '2017-05-01',
       '2017-05-08', '2017-05-15', '2017-05-22', '2017-05-29',
       '2017-06-05', '2017-06-12', '2017-06-19', '2017-06-26',
       '2017-07-03', '2017-07-10', '2017-07-17', '2017-07-24',
       '2017-07-31', '2017-08-07', '2017-08-14', '2017-08-21',
       '2017-08-28', '2017-09-04', '2017-09-11', '2017-09-18',
       '2017-09-25', '2017-10-02', '2017-10-09', '2017-10-16',
       '2017-10-23', '2017-10-30', '2017-11-06', '2017-11-13',
       '2017-11-20', '2017-11-27', '2017-12-04', '2017-12-11',
       '2017-12-18', '2017-12-25', '2018-01-01', '2018-01-08',
       '2018-01-15', '2018-01-22', '2018-01-29', '2018-02-05',
       '2018-02-12', '2018-02-19', '2018-02-26', '2018-

In [12]:
len(period_starts)

159

In [13]:
dummy = df[ (df['period_start_dt'] == '2016-12-19') | 
            (df['period_start_dt'] == '2017-12-19') | 
            (df['period_start_dt'] == '2018-12-19') |
            (df['period_start_dt'] == '2019') |

            (df['period_start_dt'] == '2016-12-26') |
            (df['period_start_dt'] == '2017-12-26') |
            (df['period_start_dt'] == '2018-12-26') |
            (df['period_start_dt'] == '2019-12-26') |

            (df['period_start_dt'] == '2016-03-06') |
            (df['period_start_dt'] == '2017-03-06') |
            (df['period_start_dt'] == '2018-03-06') |
            (df['period_start_dt'] == '2019-03-06') |

            (df['period_start_dt'] == '2016-02-20') |
            (df['period_start_dt'] == '2017-02-20') |
            (df['period_start_dt'] == '2018-02-20') |
            (df['period_start_dt'] == '2019-02-20')  ]

In [14]:
df['holiday_flag'] = np.where(df.index.isin(dummy.index), 1, 0)

In [15]:
df[df.index.isin(dummy.index)]

,Unnamed: 0,product_rk,store_location_rk,period_start_dt,demand,PROMO1_FLAG,PROMO2_FLAG,PRICE_REGULAR,PRICE_AFTER_DISC,NUM_CONSULTANT,AUTORIZATION_FLAG,date,holiday_flag
0,0,40369,309,2016-12-19,29.0,NaN,NaN,NaN,NaN,NaN,NaN,17154.0,1
1,1,40370,309,2016-12-19,64.0,NaN,NaN,NaN,NaN,NaN,NaN,17154.0,1
2,2,40372,309,2016-12-19,32.0,NaN,NaN,NaN,NaN,NaN,NaN,17154.0,1
3,3,40373,309,2016-12-19,10.0,NaN,NaN,NaN,NaN,NaN,NaN,17154.0,1
4,4,46272,309,2016-12-19,15.0,NaN,NaN,NaN,NaN,NaN,NaN,17154.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
32121,32296,40369,1326,2017-03-06,416.0,0.0,0.0,500.0,500.0,0.0,1.0,17231.0,1
32122,32297,40370,1326,2017-03-06,428.0,0.0,0.0,1000.0,1000.0,0.0,1.0,17231.0,1
32123,32298,40372,1326,2017-03-06,129.0,0.0,0.0,2000.0,2000.0,0.0,1.0,17231.0,1
32124,32299,40373,1326,2017-03-06,96.0,0.0,0.0,3000.0,3000.0,0.0,1.0,17231.0,1


In [16]:
df = df.fillna(0)

In [17]:
df

,Unnamed: 0,product_rk,store_location_rk,period_start_dt,demand,PROMO1_FLAG,PROMO2_FLAG,PRICE_REGULAR,PRICE_AFTER_DISC,NUM_CONSULTANT,AUTORIZATION_FLAG,date,holiday_flag
0,0,40369,309,2016-12-19,29.0,0.0,0.0,0.00,0.0,0.0,0.0,17154.0,1
1,1,40370,309,2016-12-19,64.0,0.0,0.0,0.00,0.0,0.0,0.0,17154.0,1
2,2,40372,309,2016-12-19,32.0,0.0,0.0,0.00,0.0,0.0,0.0,17154.0,1
3,3,40373,309,2016-12-19,10.0,0.0,0.0,0.00,0.0,0.0,0.0,17154.0,1
4,4,46272,309,2016-12-19,15.0,0.0,0.0,0.00,0.0,0.0,0.0,17154.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
35339,35537,40370,1380,2019-12-30,0.0,0.0,0.0,1000.00,1000.0,0.0,1.0,18260.0,0
35340,35538,40372,1380,2019-12-30,0.0,0.0,0.0,2000.00,2000.0,0.0,1.0,18260.0,0
35341,35539,40373,1380,2019-12-30,0.0,0.0,0.0,3000.00,3000.0,0.0,1.0,18260.0,0
35342,35540,46272,1380,2019-12-30,0.0,1.0,0.0,284.29,199.0,0.0,1.0,18260.0,0


In [18]:
jan2020 = df[df['period_start_dt'].isin(['2019-12-30'])]

In [19]:
jan2020.shape

(240, 13)

In [20]:
jan2020_df = jan2020[['period_start_dt', 'demand', 'date', 'product_rk', 'store_location_rk']]

In [21]:
df = df.drop('period_start_dt', axis=1)


In [22]:
uni_st_ids = df.store_location_rk.unique()
uni_sku_ids = df.product_rk.unique()

In [23]:
def split_dataset(df, store_ids, sku_ids, univar=False):
    batches = []
    for sku_id in sku_ids:
        for store_id in store_ids:
            if univar:
                batch = df[(df['store_location_rk'] == store_id) & (df['product_rk'] == sku_id)]['demand']
            else:
                batch = df[(df['store_location_rk'] == store_id) & (df['product_rk'] == sku_id)]
            batches.append(batch.values)
    return batches


def split_dataset_other(df, sku_ids, demand_bool=False):
    batches = []
    for sku_id in sku_ids:
        batch = df[df['product_rk'] == sku_id]['demand']
        batches.append(batch.values)
    return np.array(batches)

In [24]:
splitted_df = split_dataset(df, uni_st_ids, uni_sku_ids)

In [25]:
for sh in split_dataset_other(df, uni_sku_ids):
    print(sh.shape)

(6074,)
(6074,)
(6073,)
(6074,)
(6068,)
(4981,)


In [26]:
len(splitted_df)

246

In [27]:
len(uni_sku_ids)*len(uni_st_ids)

246

In [28]:
tmp = []
for idf in splitted_df:
    if idf.shape[0] == 159:
        tmp.append(idf)
splitted_df=tmp

In [29]:
norm_sku_store = []
for idf in splitted_df:
    norm_sku_store.append(str(idf[0, 0])+str(idf[0, 1]))
# norm_sku_store

In [30]:
len(norm_sku_store)
# norm_sku_store

170

In [31]:
jan2020_df['code'] = jan2020_df['product_rk'].astype(str) + jan2020_df['store_location_rk'].astype(str)
jan2020_df

/usr/lib/python3/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,period_start_dt,demand,date,product_rk,store_location_rk,code
932,2019-12-30,0.0,18260.0,40369,317,40369317
933,2019-12-30,0.0,18260.0,40370,317,40370317
934,2019-12-30,0.0,18260.0,40372,317,40372317
935,2019-12-30,0.0,18260.0,40373,317,40373317
936,2019-12-30,0.0,18260.0,46272,317,46272317
...,...,...,...,...,...,...
35339,2019-12-30,0.0,18260.0,40370,1380,403701380
35340,2019-12-30,0.0,18260.0,40372,1380,403721380
35341,2019-12-30,0.0,18260.0,40373,1380,403731380
35342,2019-12-30,0.0,18260.0,46272,1380,462721380


In [32]:
jan_actual_df = jan2020_df[jan2020_df['code'].isin(norm_sku_store)]

In [33]:
#WARNING:deprecated!
def create_one_hot(df, col_name):
    uni_cols = df[col_name].unique()
    df = pd.merge(df, pd.get_dummies(df[col_name], prefix=col_name), left_index=True, right_index=True)
    return df.drop(columns=col_name)

# df = create_one_hot(df, 'Store_id')
# df = create_one_hot(df, 'SKU_id')

In [36]:
def make_ds(dataset, target_index=2, history_size=5, target_size=5, start_index=0, end_index=None):
  data = []
  labels = []

  start_index = start_index + history_size
  if end_index is None:
    end_index = len(dataset) - target_size

  for i in range(start_index, end_index):
    indices = range(i-history_size, i)
    data.append(dataset[indices])
    # Reshape data from (history_size,) to (history_size, 1)
    # data.append(np.reshape(dataset[indices], (history_size, 1)))
    labels.append(dataset[i: i+target_size, target_index])
  return np.array(data), np.array(labels)

In [37]:
x, y = make_ds(splitted_df[0])


In [38]:
x.shape

(149, 5, 12)

In [39]:
y.shape

(149, 5)

In [40]:
print ('Single window of past history')
print (x[0])
print ('\n Target demands to predict')
print (y[0])

Single window of past history
[[1.5000000e+01 4.0369000e+04 3.1700000e+02 5.0000000e+01 0.0000000e+00
  0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
  1.7154000e+04 1.0000000e+00]
 [2.0000000e+01 4.0369000e+04 3.1700000e+02 6.5000000e+01 1.0000000e+00
  0.0000000e+00 5.0000000e+02 5.0000000e+02 0.0000000e+00 1.0000000e+00
  1.7161000e+04 1.0000000e+00]
 [2.5000000e+01 4.0369000e+04 3.1700000e+02 4.1251366e+01 1.0000000e+00
  0.0000000e+00 5.0000000e+02 5.0000000e+02 0.0000000e+00 1.0000000e+00
  1.7168000e+04 0.0000000e+00]
 [3.0000000e+01 4.0369000e+04 3.1700000e+02 1.3000000e+01 0.0000000e+00
  0.0000000e+00 5.0000000e+02 5.0000000e+02 0.0000000e+00 1.0000000e+00
  1.7175000e+04 0.0000000e+00]
 [3.5000000e+01 4.0369000e+04 3.1700000e+02 1.1000000e+01 0.0000000e+00
  0.0000000e+00 5.0000000e+02 5.0000000e+02 0.0000000e+00 1.0000000e+00
  1.7182000e+04 0.0000000e+00]]

 Target demands to predict
[317. 317. 317. 317. 317.]


In [41]:
def create_time_steps(length):
  return list(range(-length, 0))

In [42]:
def multi_step_plot(history, true_future, prediction):
  plt.figure(figsize=(12, 6))
  num_in = create_time_steps(len(history))
  num_out = len(true_future)

  plt.plot(num_in, np.array(history[:, 1]), label='History')
  plt.plot(np.arange(num_out)/STEP, np.array(true_future), 'bo',
           label='True Future')
  if prediction.any():
    plt.plot(np.arange(num_out)/STEP, np.array(prediction), 'ro',
             label='Predicted Future')
  plt.legend(loc='upper left')
  plt.show()

In [43]:
def show_plot(history, true_vals, preds=None):
    plt.figure(figsize=(24, 12))
    plt.xlim((0, len(history) + len(true_vals)))

    plt.plot(range(len(history)), history, label='History')
    plt.plot(range(len(history), len(history) + len(true_vals)), true_vals,  '.-', label='True')
    plt.plot(range(len(history), len(history) + len(true_vals)), preds, 'ro', label='Pred')
    # plt.xticks(labels=df.period_start_dt.unique()[:143])

    plt.legend(loc='upper left')
    plt.show()

In [44]:
def baseline(history):
  return np.mean(history)

In [45]:
x, y = make_ds(splitted_df[0], target_size=12)


In [46]:
new_df = []
for df in splitted_df:
    if df.shape[0] == 159:
        new_df.append(df)

In [47]:
model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(120, input_shape=x[0].shape[-2:], return_sequences=True),
    tf.keras.layers.LSTM(120, input_shape=x[0].shape[-2:]),
    tf.keras.layers.Dense(100),
    tf.keras.layers.ReLU(),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(12),
    tf.keras.layers.Dense(20),
    tf.keras.layers.Dense(5)
])

model.compile(optimizer='adam', loss='mape')

In [57]:
def train(df1):
    hist = []
    for df in df1:
        x1, y1 = make_ds(df, target_size=1)
        hist.append(model.train_on_batch(x1, y1))
    return hist

In [58]:
from tensorflow.keras.utils import  plot_model
plot_model(model, to_file='model.png', show_shapes=True)

('Failed to import pydot. You must `pip install pydot` and install graphviz (https://graphviz.gitlab.io/download/), ', 'for `pydotprint` to work.')


In [59]:
hist = train(new_df)

UnknownError:    Fail to find the dnn implementation.
	 [[{{node CudnnRNN}}]]
	 [[sequential/lstm/PartitionedCall]] [Op:__inference_train_function_5123]

Function call stack:
train_function -> train_function -> train_function


In [55]:
from __future__ import absolute_import, division, print_function, unicode_literals
import os
import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow.python.client import device_lib

dataset, info = tfds.load('imdb_reviews/subwords8k', with_info=True,
                          as_supervised=True)
train_dataset, test_dataset = dataset['train'], dataset['test']

BUFFER_SIZE = 10000
BATCH_SIZE = 64

train_dataset = train_dataset.shuffle(BUFFER_SIZE)
train_dataset = train_dataset.padded_batch(BATCH_SIZE, train_dataset.output_shapes)

test_dataset = test_dataset.padded_batch(BATCH_SIZE, test_dataset.output_shapes)
encoder = info.features['text'].encoder


model = tf.keras.Sequential([
    tf.keras.layers.Embedding(encoder.vocab_size, 64),
    #tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])
history = model.fit(train_dataset, epochs=10,
                    validation_data=test_dataset, 
                    validation_steps=30)

Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


KeyboardInterrupt: 